# Setting up Notebook

In [1]:
# Importing Relevant Packages
import numpy as np
import pandas as pd

from pprint import pprint
import re
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
# Importing Datasets
video_comments_df = pd.read_csv("/home/chigoz/Downloads/Disso/00. Inputs/Video_Comments_Table.csv", delimiter = ";")
text_df = pd.read_csv("/home/chigoz/Downloads/Disso/00. Inputs/text_data.csv", delimiter = ",")

# Keyword Extraction

In [3]:
# Defining function to preprocess text data
def pre_process(text):
    
    # convert all characters to lowercase
    text=text.lower()
    
    #remove urls
    text=re.sub(r'http\S+', '', text)
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

# Combining the video title and description fields 
text_df['kw_text'] = text_df['video_title'] + text_df['video_description']

# Applying the pre processing to the combined text field
text_df['kw_text'] = text_df['kw_text'].apply(lambda x:pre_process(x))
text_df.head()

video_id                                        video_title  \
0  3Cg8-eoNXdY  SLIM FIT DENIM JEANS LATEST PICK UPS + COLLECT...   
1  zgXoCGUcR9w  ☀️☀️SUMMER OUTFIT PICKUPS UNBOXING | TMONCLER,...   
2  AFpME2E2GAY  MONTHLY DESIGNER SELECTIONS UNBOXING & TRY-ON ...   
3  4TJCzY3Ph88  ULTIMATE BALENCIAGA TRIPLE S , TRACK & SPEED S...   
4  TX3TFXp2y2M  🔥🔥 6 HOT MENSWEAR SUMMER SALES HAPPENING RIGHT...   

                                   video_description  \
0  This video has been sponsored by @FarFetch\nGE...   
1  THIS VIDEO HAS BEEN SPONSORED BY @SEVEN STORE\...   
2  THIS VIDEO HAS BEEN SPONSORED BY LUISAVIAROMA\...   
3  This video has been sponsored @Browns \n@Brown...   
4  https://thehoxtontrend.com/latest-spring-sales...   

                                          video_tags  \
0  ['mens fashion', 'Denim Jeans', 'Slim fit deni...   
1  ['mens fashion', 'MENS SUMMER OUTFITS', 'MENS ...   
2  ['mens fashion', 'tkmaxx outfit challenge', 's...   
3  ['mens fashion', 'tkmaxx outfit challenge', 's...   
4  ['mens fashion', 'tkmaxx outfit challenge', 's...   

                                      video_comments  \
0  ['GET AN EXTRA 15% OFF SALE ITEMS  @FARFETCH  ...   
1  ["GET 15% OFF WITH CODE 'THT15' WORKS ON NEW S...   
2  ['I like that orange moncler jacket!', "My fav...   
3  ['ENTER OUR 250 COMPETITION GIVEAWAY TO SPEND ...   
4  ['Reppin the Patta collab! ', 'Thanks picking ...   

                                             kw_text  
0  slim fit denim jeans latest pick ups collectio...  
1   summer outfit pickups unboxing tmoncler cdg n...  
2  monthly designer selections unboxing try on ft...  
3  ultimate balenciaga triple s track speed sock ...  
4   hot menswear summer sales happening right now...

In [4]:
# Setting lists of stop words
stopwords=["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz", "year", "day", "month", "yesterday"]

#get the text column 
docs=text_df['kw_text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,
                   stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

/home/chigoz/anaconda3/envs/ml_module/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['articl', 'mon'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [5]:
# Shows there is 210 documents (video titles/descriptions) and vocab of 2118 after the countvectoriser
word_count_vector.shape

(210, 1772)

In [6]:
# Setting parameters and fitting tf idf transformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [7]:
# Creating a function which perserves the column index while soting the vector values
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

# Creating a function to get the feature value names and scores
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
    
    return feature_vals

In [8]:
# Assigning column used for keyword extraction to the doc variable
doc=text_df['kw_text']

# Getting feature names from matrix
feature_names=cv.get_feature_names()

#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(doc))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,5)
    
    # appending keywords to keyword list
    results.append(keywords)

In [9]:
# Creating a keyword results dataframe
results_df = pd.DataFrame(results, columns=["keyword_1", "keyword_2", "keyword_3", "keyword_4", "keyword_5"])

# Creating dummy variables for each keyword
results_df = results_df.stack().str.get_dummies().sum(level=0)
results_df

# Appending results of a keyword dataframe to an overall NLP dataframe
nlp_df = pd.concat([text_df["video_id"], results_df], axis=1)
nlp_df.head()

video_id  accounts  adidas  ages  air  alexander  alligator  alpha  \
0  3Cg8-eoNXdY         0       0     0    0          0          0      0   
1  zgXoCGUcR9w         0       0     0    0          0          0      0   
2  AFpME2E2GAY         0       0     0    0          0          0      0   
3  4TJCzY3Ph88         0       0     0    0          0          0      0   
4  TX3TFXp2y2M         0       0     0    0          0          0      0   

   alternatives  alyx  ...  winner  winter  wnjoy  wol  worldwide  worth  \
0             0     0  ...       0       0      0    0          0      0   
1             0     0  ...       0       0      0    0          0      0   
2             0     0  ...       0       0      0    0          0      0   
3             0     0  ...       0       0      0    0          0      0   
4             0     0  ...       0       0      0    0          0      0   

   wotherspoon  wyndham  yeezy  zee  
0            0        0      0    0  
1            0        0      0    0  
2            0        0      0    0  
3            0        0      0    0  
4            0        0      0    0  

[5 rows x 577 columns]

# Sentiment Analysis

In [10]:
# Defining function to preprocess text data
def pre_process(text):
    
    # Remove Non-Letters
    # (Search for non letters and replace with spaces)
    text=re.sub("[^a-zA-Z]"," ",str(text))
    
    #remove urls
    text=re.sub(r'http\S+', '', text)
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

# Preprocessing comments
all_comments = list(video_comments_df["comment"].apply(lambda x:pre_process(x)))

# Empty list for each comment's sentiment score
sentiment_scores = []

# Setting up sentiment analyser function
sia = SentimentIntensityAnalyzer()

# Creating a list containing all the levels within the comments column of the comments dataframe
levels = list(range(0, len(video_comments_df)))

# For loop to analyse all the comments 
for comment in levels:
    
    score = sia.polarity_scores(all_comments[comment])["compound"]
    
    sentiment_scores.append(score)

In [11]:
# Adding sentiment score column to the comments dataframe
video_comments_df["sentiment_score"] = sentiment_scores

# Calculating the mean sentiment score for each video
avg_sentiment_scores = video_comments_df[['video_id', 'sentiment_score']].groupby('video_id').mean()
avg_sentiment_scores.reset_index(inplace = True)

In [12]:
# Merging the sentiment scores onto the nlp data dataframe
nlp_df = nlp_df.merge(avg_sentiment_scores, on = "video_id")
nlp_df.head()

video_id  accounts  adidas  ages  air  alexander  alligator  alpha  \
0  3Cg8-eoNXdY         0       0     0    0          0          0      0   
1  zgXoCGUcR9w         0       0     0    0          0          0      0   
2  AFpME2E2GAY         0       0     0    0          0          0      0   
3  4TJCzY3Ph88         0       0     0    0          0          0      0   
4  TX3TFXp2y2M         0       0     0    0          0          0      0   

   alternatives  alyx  ...  winter  wnjoy  wol  worldwide  worth  wotherspoon  \
0             0     0  ...       0      0    0          0      0            0   
1             0     0  ...       0      0    0          0      0            0   
2             0     0  ...       0      0    0          0      0            0   
3             0     0  ...       0      0    0          0      0            0   
4             0     0  ...       0      0    0          0      0            0   

   wyndham  yeezy  zee  sentiment_score  
0        0      0    0         0.358728  
1        0      0    0         0.211008  
2        0      0    0         0.325584  
3        0      0    0         0.213441  
4        0      0    0         0.167957  

[5 rows x 578 columns]

In [13]:
# Saving Dataframe to CSV File
nlp_df.to_csv("/home/chigoz/Downloads/Disso/00. Inputs/nlp_data.csv", index=False, sep = ',')